In [1]:
import pandas as pd
import sys
import os.path
import numpy as np
import cProfile
import pstats
import bokeh.plotting
from bokeh.plotting import figure, output_notebook, show, gridplot, reset_output

sys.path.append("/home/daliu/Src/spykshrk_realtime/")

import spykshrk.realtime.binary_record as bin_rec

pd.set_option('float_format', '{:,.1f}'.format)
pd.set_option('display.max_rows', 10)
pd.set_option('display.width', 120)

output_notebook()

In [4]:
rec_dir = '/opt/data36/daliu/realtime/spykshrk/timing'
# rec_dir = '/home/daliu/output/test'
mpi_ranks = [0, 2, 3]
file_prefix = 'test_animal'
manager_label = 'state'
file_postfix = 'bin_rec'

allrecs = {}

for mpi_rank in mpi_ranks:
    binrec = bin_rec.BinaryRecordsFileReader(save_dir=rec_dir, file_prefix=file_prefix,
                                             mpi_rank=mpi_rank,
                                             manager_label=manager_label,
                                             file_postfix=file_postfix, filemeta_as_col=False)

    rec_pandas = binrec.convert_pandas()
    
    for rec_id, rec_pd in rec_pandas.items():
        existing_rec_pd = allrecs.setdefault(rec_id, pd.DataFrame())
        allrecs[rec_id] = existing_rec_pd.append(rec_pd, ignore_index=True)
    
for rec_id, rec_pd in allrecs.items():
    if 'timestamp' in rec_pd.columns:
        rec_pd.sort_values(['timestamp']).reset_index(drop=True)


In [5]:
allrecs.keys()

In [31]:
timing = allrecs[100]
single_timing_example = timing[(timing['timestamp'] == 73830243) & (timing['ntrode_id'] == 5)]

single_timing_example[single_timing_example['label'] == 'stim_rip_state']['wtime'].values - \
    single_timing_example[single_timing_example['label'] == 'rip_recv']['wtime'].values


In [32]:
timing.sort_values(by=['timestamp', 'ntrode_id', 'wtime'], inplace=True)


In [40]:
boundary_time = timing['wtime'][timing['label'] == 'stim_rip_state'].values - timing['wtime'][timing['label'] == 'rip_recv'].values

In [44]:
pl

In [20]:
ret_bytes = allrecs[100]['label'].iloc[0]
ret_bytes.split(b'\0')[0]

In [6]:
ripple_ntrode5

In [14]:
s1 = figure(plot_width=900, plot_height=400)
s1.line(x=ripple_ntrode5['current_time']/30000,
       y=ripple_ntrode5['lfp_data'], color='gray')
s1.line(x=ripple_ntrode5['current_time']/30000,
       y=ripple_ntrode5['rd'], color='blue')
s1.line(x=ripple_ntrode5['current_time']/30000,
       y=ripple_ntrode5['current_val'], color='red')


show(s1)